In [1]:
!pip install -q transformers

/bin/bash: /home/anwer/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [2]:
import torch
from transformers import BertModel, BertTokenizer
import re
import os
import requests
from tqdm.auto import tqdm

In [3]:
tokenizer = BertTokenizer.from_pretrained('Rostlab/prot_bert_bfd', do_lower_case=False )

In [4]:
model = BertModel.from_pretrained("Rostlab/prot_bert_bfd")

Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [16]:
model = model.to(device)
model = model.eval()

In [90]:
sequences_Example = ["A E T C Z A O","S K T Z P S T P S A E T"]
# sequences_Example = ["A E T C Z A O"]
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]
sequences_Example

['A E T C X A X', 'S K T X P S T P S A E T']

In [99]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True)

In [100]:
ids

{'input_ids': [[2, 6, 9, 15, 23, 25, 6, 25, 3, 0, 0, 0, 0, 0], [2, 10, 12, 15, 25, 16, 10, 15, 16, 10, 6, 9, 15, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [101]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

In [102]:
with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)[0]

In [103]:
embedding = embedding.cpu().numpy()

In [104]:
b = embedding[0][0]
a == b 

array([False, False, False, ..., False, False, False])

In [105]:
a

array([ 0.07873306,  0.02530314,  0.10022404, ..., -0.11921907,
       -0.08556504,  0.01579634], dtype=float32)

In [106]:
b

array([ 0.07873297,  0.02530355,  0.1002248 , ..., -0.11921974,
       -0.08556592,  0.01579616], dtype=float32)